В интернет-галерее пользователи могут просматривать тысячи картинок произвольного содержания. При этом они совершают различные действия, например кликают на картинку чтобы увеличить её в размере или отложить её в избранное. Имея такую информацию, часто можно предугадать дальнейшие действия пользователя, на чём и основано большинство современных систем рекомендаций.

В этой задаче вам предлагается создать свою собственную систему рекомендаций.

Имея идентификаторы картинок, с которыми пользователи взаимодействовали, вам нужно предсказать, на какие картинки пользователи вероятнее всего кликнут. При этом картинки, на которые пользователи совершали клики в прошлом, не будут участвовать в оценке, даже если на них будет повторно совершён клик в будущем. Для предсказания множество картинок включает в себя все идентификаторы, которые представлены в файлах.



In [1]:
import pandas as pd
import numpy as np

from scipy.sparse import coo_matrix

import implicit

In [2]:
def load_df(filename, use_days=False):
    data = pd.read_csv('data/' + filename)
    if use_days:
        data.day = pd.to_datetime(data.day)
        data.day = data.day.max() - data.day
        data.day = (data.day / np.timedelta64(1, 'D')).astype(int)
        
    return data

def describe_df(data, name):
    print('------------{}------------'.format(name))
    print(data.shape)
    print(data.head())
    if 'day' in data.keys():
        print('last day is {}'.format(np.max(data['day'])))

def apk(actual, predicted, k=100): 
    """ 
    Computes the average precision at k. 
    This function computes the average prescision at k between two lists of 
    items. 
    Parameters 
    ---------- 
    actual : list 
             A list of elements that are to be predicted (order doesn’t matter) 
    predicted : list 
                A list of predicted elements (order does matter) 
    k : int, optional 
        The maximum number of predicted elements 
    Returns 
    ------- 
    score : double 
            The average precision at k over the input lists 
    """ 
    if len(predicted)>k: 
        predicted = predicted[:k] 
 
    score = 0.0 
    num_hits = 0.0 
 
    for i,p in enumerate(predicted): 
        if p in actual and p not in predicted[:i]: 
            num_hits += 1.0 
            score += num_hits / (i+1.0) 
 
    if not actual: 
        return 0.0 
 
    return score / min(len(actual), k) 

def mapk(actual, predicted, k=100): 
    """ 
    Computes the mean average precision at k. 
    This function computes the mean average prescision at k between two lists 
    of lists of items. 
    Parameters 
    ---------- 
    actual : list 
             A list of lists of elements that are to be predicted 
             (order doesn’t matter in the lists) 
    predicted : list 
                A list of lists of predicted elements 
                (order matters in the lists) 
    k : int, optional 
        The maximum number of predicted elements 
    Returns 
    ------- 
    score : double 
            The mean average precision at k over the input lists 
    """ 
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [9]:
def create_st(data_frames, max_cost):
    """
    Create sparse table with different weights
    """
    
    data = []
    X = []
    Y = []
    
    for df in data_frames:
        print(type(df))
        users = np.array(df.user_id)
        items = np.array(df.picture_id)
        days = np.array(df.day)
        weight = max_cost - days / 51. * (max_cost - 1)
        
        data.append(weight)
        X.append(users)
        Y.append(items)
        print('Complete')
        
    data = np.hstack(data)
    X = np.hstack(X)
    Y = np.hstack(Y)
    
    return coo_matrix((data, (X, Y)))

# Анализ данных

* ada

 ## Считываем данные

In [4]:
clicks = load_df('train_clicks.csv', True)
likes = load_df('train_likes.csv', True)
shares = load_df('train_shares.csv', True)
descriptions = load_df('descriptions.csv')
themes = load_df('themes.csv')
user_proﬁles = load_df('user_information.csv')
test_users = load_df('test_users.csv')

##  Выводим информацию

In [5]:
describe_df(clicks, 'clicks')
describe_df(likes, 'likes')
describe_df(shares, 'shares')
describe_df(descriptions, 'descriptions')
describe_df(themes, 'themes')
describe_df(user_proﬁles, 'user_proﬁles')

------------clicks------------
(2360862, 3)
   user_id  picture_id  day
0     1442      546149   43
1     1442      546149   43
2     1442     1242875   36
3     1442     1242875   36
4     1442     1242891   36
last day is 51
------------likes------------
(308422, 3)
   user_id  picture_id  day
0   490982      298754    3
1   490989     1060978   35
2   490989      976274   35
3   490989      976273   35
4   490989      976272   35
last day is 51
------------shares------------
(47867, 3)
   user_id  picture_id  day
0    24986      216354   37
1    25046      514369   16
2    25058     1066857   47
3    25058      481307   47
4    25058      636884   47
last day is 51
------------descriptions------------
(1379875, 2)
   picture_id                                        description
0     1171496      734475 487260 884706 5941 48255 147285 969800
1     1171497  488295 261184 466630 235887 642445 465419 9297...
2     1171498                              724435 1003828 164020
3     1171499

# Обучение модели

Создаём разреженную матрицу пользователь — объект для трёх таблиц с разными весами

In [50]:
user_item = create_st([clicks, likes, shares], 8)

<class 'pandas.core.frame.DataFrame'>
Complete
<class 'pandas.core.frame.DataFrame'>
Complete
<class 'pandas.core.frame.DataFrame'>
Complete


В качестве модели будем использовать разложение матрицы с помощью метода ALS

In [65]:
model = implicit.als.AlternatingLeastSquares(factors=100, regularization=.1, iterations=100, num_threads=4)

In [66]:
model.fit(user_item.T.tocsr())

100%|██████████| 100.0/100 [06:31<00:00,  3.83s/it]


Прочитаем идентификаторы пользователей, для которых нам нужно сделать предсказания

In [67]:
test_users = pd.read_csv('data/test_users.csv')

In [68]:
user_item_csr = user_item.tocsr()

Для каждого пользователя найдём 100 самых релевантных изображений

In [69]:
rows = []
for user_id in test_users.user_id.values:
    items = []
    for i in model.recommend(user_id, user_item_csr, N=100):
        items.append(i[0])
        #if len(rows) == 0 and i[0] in wow:
         #   print("WRONG")
    rows.append(items)

Отформатируем идентификаторы как нужно для ответа

In [70]:
test_users['predictions'] = list(map(lambda x: ' '.join(map(str, x)), rows))

И запишем предсказания в файл

In [71]:
test_users.to_csv('predictions.csv', index=False)